In [1]:
'''
TODO
if one only has suffix or prefix addition to the other one, it will be count as 100% similarity
drop vcc and vss from results
warn about vcc connected to vss
'''

'\nTODO\nif one only has suffix or prefix addition to the other one, it will be count as 100% similarity\ndrop vcc and vss from results\nwarn about vcc connected to vss\n'

In [2]:
%%time
from glob import glob
import pandas as pd
import difflib
import hashlib
import ipywidgets
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()

CPU times: user 740 ms, sys: 700 ms, total: 1.44 s
Wall time: 1.3 s


In [3]:
%matplotlib inline
pattern=r'son - (?P<son_net>[\w\d_]+)@(?P<son_block>[\w\d_]+) \((?P<son_type>[\w_-]*):.*:(?P<son_direction>[\w_-]*)\) father - (?P<father_net>[\w\d_\[\]]+)@(?P<father_block>[\w\d_]+) \((?P<father_type>[\w_-]*):'

output1=ipywidgets.Output(layout=ipywidgets.Layout(overflow='scroll', height='500px'))
output2=ipywidgets.Output(layout=ipywidgets.Layout(overflow='scroll', height='400px'))
verilog_folder_widget=ipywidgets.Text(value=r'/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/brk_gen2/4.brk_gen2_0p8/0.original/1',placeholder='verilog folder full path',
                                      description='verilog folder:', 
                                      layout=ipywidgets.Layout(width='95%', height='40px'),
                                      style=dict(description_width='initial'))
def generate_port_net_mapping(verilog_folder):
    output1.clear_output()
    output2.clear_output()

    with output1:
        parser_output_folder='/nfs/iil/disks/ams_regruns/dkl/users/lisrael1/msv/temp_del/verilog_helper/'

        # it takes a lot of time to run, so we will save old results by the folder hash and reload this result at next time
        verilog_folder=verilog_folder['new']
        verilog_folder_hash=hashlib.md5(str.encode(verilog_folder)).hexdigest()
        parser_output_file=parser_output_folder+'/perl_verilog_parser_output_'+verilog_folder_hash+'.txt'
        similarity_output_file=parser_output_file.replace('perl_','similarity_').replace('.txt','.csv')
        print('parser_output_file: {}\nsimilarity_output_file: {}'.format(parser_output_file, similarity_output_file))
        exist=!ls $parser_output_file
        if exist[0].startswith('ls: cannot access '):
            print('running parser (may take 1 minute)')
            command='/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/tools/sv/print_all_modules_and_ports_connection.pl -f='+\
                verilog_folder+' 2>&1 |pv -l -N "lines rate at reading parser output"> '+\
                parser_output_file
            print('running command:\n\t'+command)
            !$command
        else:
            print('this folder already been parsed from before. using parsed output')
        exist=!ls $similarity_output_file
        if exist[0].startswith('ls: cannot access '):
            print('parsing file '+parser_output_file)
            parser_output=open(parser_output_file).read().split('\n')
            print('done parsing')
            df=pd.DataFrame(parser_output, columns=['original'])
            print('done df')
            df=pd.concat([df.original.str.extract(pattern),df], axis=1)
            print('done parsing df. tail of this df:')
            display(df.tail(10))
        #         display(df.query('((son_block==@block) and (son_net==@net)) or ((father_block==@block) and (father_net==@net)) '))

            print('now finding similarity')
            ratio=df.progress_apply(lambda r:difflib.SequenceMatcher(a=str(r.son_net).split('[')[0].lower(), b=str(r.father_net).split('[')[0].lower()).ratio(), axis=1).round(2)*100

            df=pd.concat([ratio.to_frame('similarity'), df], axis=1)
            df['son_len']=df.dropna().son_net.apply(len)
            df['father_len']=df.dropna().father_net.apply(len)
            df.to_csv(similarity_output_file)
        else:
            print('this folder already been calculated for similarity. reading last table output')
            df=pd.read_csv(similarity_output_file, index_col=[0])
            print('done reading csv')
        # taking only names that has above 4 characters
        relevant=df.dropna(subset=['son_len','father_len'], how='any', axis=0).query('son_len>4 & father_len>4').\
            query('~son_block.str.endswith("res") & son_block!="msv_rc_filter"').sort_values('similarity')
        relevant.similarity=relevant.similarity.astype(int)
        print('relevant table:')
        display(pd.concat([relevant.head(10),relevant.tail(10)], axis=0))
    with output2:
        ax=relevant.similarity.plot.hist(bins=10,density=True,cumulative=True, title='port name vs input net name similarity')
        ax.set_xlabel("similarity")
        display(ax.figure)
        # relevant.ratio.plot.hist(bins=100,density=False,cumulative=True)
        print('some examples with different similarities')
        display(relevant[['similarity','son_net','son_block','father_net','father_block']].query('similarity<100').sample(50).sort_values('similarity'))

verilog_folder_widget.observe(generate_port_net_mapping, names='value')
grid = ipywidgets.GridspecLayout(15, 2, height='1300px', width='1400px') # note that grid 
grid[0, 0:2] = verilog_folder_widget
grid[1:6, 0:2] = output1
grid[7:14, 0:2] = output2
grid

GridspecLayout(children=(Text(value='/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/brk_gen2/4.…

In [51]:
if 0:
    similarity_output_file='/nfs/iil/disks/ams_regruns/dkl/users/lisrael1/msv/temp_del/verilog_helper//similarity_verilog_parser_output_85e5e22b0de0ca8454c70978d6f5fab8.txt'
    df=pd.read_csv(similarity_output_file, index_col=[0])
if 0:
    relevant=df.dropna(subset=['son_len','father_len'], how='any', axis=0).query('son_len>4 & father_len>4').\
        query('~son_block.str.endswith("res") & son_block!="msv_rc_filter"').sort_values('similarity')
    relevant.similarity=relevant.similarity.astype(int)
    print('relevant table:')
    display(pd.concat([relevant.head(10),relevant.tail(10)], axis=0))
#     display(relevant)

relevant table:


,similarity,son_net,son_block,son_type,son_direction,father_net,father_block,father_type,original,son_len,father_len
351112,11,i_quad_mid_en_runinv_for_pd_fw,ipn5adcsararrayshr_quad_quartet,wire,in,cdsNet5,ipn5adcsararrayshr_shared_top,wire,port_report: ipn5brk2adctop_adc_top.i...,30.0,7.0
351178,11,i_quad_up_en_runinv_for_pd_fw,ipn5adcsararrayshr_quad_quartet,wire,in,cdsNet3,ipn5adcsararrayshr_shared_top,wire,port_report: ipn5brk2adctop_adc_top.i...,29.0,7.0
347405,12,i_summer_q0_qrt2_sig_p,ipn5adcsararrayshr_shared_top,var,in,voutp_adcfe[8],ipn5brk2adctop_adc_top,real,port_report: ipn5brk2adctop_adc_top.<ip...,22.0,14.0
347425,12,i_summer_q3_qrt0_sig_p,ipn5adcsararrayshr_shared_top,var,in,voutp_adcfe[3],ipn5brk2adctop_adc_top,real,port_report: ipn5brk2adctop_adc_top.<ip...,22.0,14.0
347424,12,i_summer_q3_qrt0_sig_n,ipn5adcsararrayshr_shared_top,var,in,voutn_adcfe[3],ipn5brk2adctop_adc_top,real,port_report: ipn5brk2adctop_adc_top.<ip...,22.0,14.0
347423,12,i_summer_q2_qrt3_sig_p,ipn5adcsararrayshr_shared_top,var,in,voutp_adcfe[14],ipn5brk2adctop_adc_top,real,port_report: ipn5brk2adctop_adc_top.<ip...,22.0,15.0
347422,12,i_summer_q2_qrt3_sig_n,ipn5adcsararrayshr_shared_top,var,in,voutn_adcfe[14],ipn5brk2adctop_adc_top,real,port_report: ipn5brk2adctop_adc_top.<ip...,22.0,15.0
347421,12,i_summer_q2_qrt2_sig_p,ipn5adcsararrayshr_shared_top,var,in,voutp_adcfe[10],ipn5brk2adctop_adc_top,real,port_report: ipn5brk2adctop_adc_top.<ip...,22.0,15.0
1933853,12,sel_b,ipn5brk2dfx_mux_2to1,wire,in,common_regs_o_hsmon_dalign_en,ipn5adcsararrayshr_strbreg_channel,logic,port_report: ipn5brk2adctop_adc_top...,5.0,29.0
429198,12,o_vref0,ipn5brk2adcldo_dacldo_drv,wire,out,vssx_psub,ipn5adcsararrayshr_octet,logic,port_report: ipn5brk2adctop_adc_t...,7.0,9.0


In [ ]:
# pattern=r'son - (?P<son_net>[\w\d_]+)@(?P<son_block>[\w\d_]+) \((?P<son_type>[\w_-]*):.*:(?P<son_direction>[\w_-]*)\) father - (?P<father_net>[\w\d_\[\]]+)@(?P<father_block>[\w\d_]+) \((?P<father_type>[\w_-]*):'
# parser_output_folder='/nfs/iil/disks/ams_regruns/dkl/users/lisrael1/msv/temp_del/verilog_helper/'
# parser_output=parser_output_folder+'/perl_verilog_parser_output_*.txt'
# !ls -l $parser_output
# parser_output='/nfs/iil/disks/ams_regruns/dkl/users/lisrael1/msv/temp_del/verilog_helper//perl_verilog_parser_output_85e5e22b0de0ca8454c70978d6f5fab8.txt'

In [ ]:
# %%time
# parser_output=open(parser_output).read().split('\n')
# print('done parsing')
# df=pd.DataFrame(parser_output, columns=['original'])
# print('done df')
# df=pd.concat([df.original.str.extract(pattern),df], axis=1)
# print('done parsing df')